本文内容主要来自：
1. https://www.ibm.com/developerworks/cn/linux/l-slurm-utility/index.html
2. http://www.vpsee.com/2013/02/install-slurm-on-ubuntu/
3. https://slurm.schedmd.com

## 简介
SLURM 是一种可用于大型计算节点集群的高度可伸缩和容错的集群管理器和作业调度系统(类似 Sun Grid Engine (SGE))。SLURM 维护着一个待处理工作的队列并管理此工作的整体资源利用。它还以一种排他或非排他的方式管理可用的计算节点（取决于资源的需求）。最后，SLURM 将作业分发给一组已分配的节点来执行工作并监视平行作业至其完成。<br>
本质上，SLURM 是一个强健的集群管理器（更关注于对功能丰富性的需求方面），它高度可移植、可伸缩至大型节点集群、容错好，而且更重要的是它是开源的。<br>
SLURM 通过一对冗余集群控制节点（冗余是可选的）来管理集群计算节点，是由一个名为 slurmctld 的管理守护程序实现的，slurmctld 提供了对计算资源的监视、分配和管理，并将进入的作业序列映射和分发到各个计算节点上。每个计算节点也有一个守护程序 slurmd，slurmd 管理在其上运行的节点，监视节点上运行的任务、接受来自控制节点的请求和工作、将工作映射到节点内部等等。图示如下：<br>
![slurm1](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/slurm1.png)
![slurm3](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/slurm3.png)
此架构内还存在其他的守护程序，比如，实现安全的身份验证。但是集群并不仅仅是节点的随机组合，因为这些节点可以是逻辑相关的，以适时实现平行计算。
一组节点也可以组成一个逻辑组，称为分区，分区通常会包含进入工作的队列。分区也可以配置各种约束条件，比如哪个用户可以使用它，分区支持的时限的作业大小。分区的更进一步优化，就是将分区内的一组节点在工作的一段时间内映射到一个用户，这就是一个作业。一个作业内，是一个或多个作业步骤，即在节点子集上执行的任务集。
下图展示了这个层次结构，进一步说明了资源的 SLURM 分区。请注意，这种分区包含了对资源的感知，相当于确保协作节点间的低延迟通信。
![slurm2](https://raw.githubusercontent.com/HuangYiran/readTech/master/fotos/slurm2.png)


## Slurm的基本命令
##### 启动
要启动 SLURM，只需使用 /etc/init.d/slurm 内定义的管理脚本。此脚本接受 start、stop、restart 和 startclean（以忽略之前保存的所有状态）。用这种方法启动 SLURM 会导致 slurmctld 守护程序的启动（在这个简单配置中，还包括您节点上的 slurmd 守护程序）<br>
<b>$ sudo /etc/init.d/slurm-llnl start</b> <br>
为了验证 SLURM 是否在运行，可以使用 sinfo 命令。sinfo 命令会返回有关这些 SLURM 节点和分区的信息

##### scontrol
用 scontrol 命令可以获得更多信息，这就使您可以查看集群各方面的详细信息<br>

$ scontrol show partition
PartitionName=debug
   AllocNodes=ALL AllowGroups=ALL Default=YES
   DefaultTime=NONE DisableRootJobs=NO Hidden=NO
   MaxNodes=UNLIMITED MaxTime=UNLIMITED MinNodes=1
   Nodes=mtj-VirtualBox
   Priority=1 RootOnly=NO Shared=NO PreemptMode=OFF
   State=UP TotalCPUs=1 TotalNodes=1
 
$ scontrol show node mtj-VirtualBox
NodeName=mtj-VirtualBox Arch=i686 CoresPerSocket=1
   CPUAlloc=0 CPUErr=0 CPUTot=1 Features=(null)
   Gres=(null)
   OS=Linux RealMemory=1 Sockets=1
   State=IDLE ThreadsPerCore=1 TmpDisk=0 Weight=1
   BootTime=2012-03-07T14:59:01 SlurmdStartTime=2012-04-17T11:10:43
   Reason=(null)

##### srun
srun命令可以为您的作业分配一个计算资源并启动一个任务。请注意您也可以分别实现这两个目的（通过 salloc 和 sbatch）。如下所示，您可以提交一个简单的 shell 命令作为您的作业来演示 srun，然后再提交一个 sleep 命令（带参数）来演示 squeue 命令的使用，从而展示集群内存在的作业。<br>
- salloc is used to allocate resources for a job in real time. Typically this is used to allocate resources and spawn a shell. The shell is then used to execute srun commands to launch parallel tasks.
- sbatch is used to submit a job script for later execution. The script will typically contain one or more srun commands to launch parallel tasks.
- squeue reports the state of jobs or job steps. It has a wide variety of filtering, sorting, and formatting options. By default, it reports the running jobs in priority order and then the pending jobs in priority order.

$ srun -l hostname
0: mtj-VirtualBox
$ srun -l sleep 5 &
[1] 24127
$ squeue
  JOBID PARTITION     NAME     USER  ST       TIME  NODES NODELIST(REASON)
     15     debug    sleep      mtj   R       0:03      1 mtj-VirtualBox
$
[1]+  Done                    srun -l sleep 5
$

注意，向集群提交的作业可以是一个简单的 Linux 命令、一个 shell 脚本文件或一个适当的可执行文件。
##### scancel
让我们来看看如何停止一个作业。在本例中，您启动一个运行较长的作业并使用 squeue 来识别其 ID。然后，使用 scancel 命令与这个作业 ID 来终止该作业步骤

$ srun -l sleep 60 &
[1] 24262
$ squeue
  JOBID PARTITION     NAME     USER  ST       TIME  NODES NODELIST(REASON)
     16     debug    sleep      mtj   R       0:03      1 mtj-VirtualBox
$ scancel 16
srun: Force Terminated job 16
$ srun: Job step aborted: Waiting up to 2 seconds for job step to finish.
0: slurmd[mtj-VirtualBox]: error: *** STEP 16.0 CANCELLED AT 2012-04-17T12:08:08 ***
srun: error: mtj-VirtualBox: task 0: Terminated
 
[1]+  Exit 15                 srun -l sleep 60
$

##### sacct 
is used to report job or job step accounting information about active or completed jobs.
##### sattach 
is used to attach standard input, output, and error plus signal capabilities to a currently running job or job step. One can attach to and detach from jobs multiple times.
##### sbcast 
is used to transfer a file from local disk to local disk on the nodes allocated to a job. This can be used to effectively use diskless compute nodes or provide improved performance relative to a shared file system.
##### sinfo 
reports the state of partitions and nodes managed by Slurm. It has a wide variety of filtering, sorting, and formatting options.
##### smap 
reports state information for jobs, partitions, and nodes managed by Slurm, but graphically displays the information to reflect network topology.
##### strigger 
is used to set, get or view event triggers. Event triggers include things such as nodes going down or jobs approaching their time limit.
##### sview 
is a graphical user interface to get and update state information for jobs, partitions, and nodes managed by Slurm.

##### 停止集群

$ sudo /etc/init.d/slurm-llnl stop
 * Stopping slurm central management daemon slurmctld                           [ OK ]
 * Stopping slurm compute node daemon slurmd                                    [ OK ]
slurmd is stopped
$

## example

此例来自https://slurm.schedmd.com/quickstart.html<br>
##### sinfo
首先我们通过sinfo命令，了解在这个系统当前的状态以及其上都有哪几个分区，他们各自包括了哪些节点。

adev0: sinfo
PARTITION AVAIL  TIMELIMIT NODES  STATE NODELIST
debug*       up      30:00     2  down* adev[1-2]
debug*       up      30:00     3   idle adev[3-5]
batch        up      30:00     3  down* adev[6,13,15]
batch        up      30:00     3  alloc adev[7-8,14]
batch        up      30:00     4   idle adev[9-12]

这里我们可以知道，我们有两个分区：debug和batch。debug名字后面的星号点明了这个分区是默认分区。为了呈现每个节点的状态，相同分区的信息被划分为多个行被展现出来。在这里可以看到adev[1-2]的状态时down。down后面的星星点明这些节点不会对请求做出回应。<br>
Note the use of a concise expression for node name specification with a common prefix adev and numeric ranges or specific numbers identified. This format allows for very clusters to be easily managed. The sinfo command has many options to easily let you view the information of interest to you in whatever format you prefer. See the man page for more information.<br>
#### squeue
下面我们通过squeue看一下目前系统上都有哪些jobs:

adev0: squeue
JOBID PARTITION  NAME  USER ST  TIME NODES NODELIST(REASON)
65646     batch  chem  mike  R 24:19     2 adev[7-8]
65647     batch   bio  joan  R  0:09     1 adev14
65648     batch  math  phil PD  0:00     6 (Resources)

- 其中的ST列，指代的是job的状态：R代表的是running，PD则是pending。可以看到这里又两个jobs是正在运行的。
- Time指出了这个job当前已经使用了的时间，他的格式是：days-hours:minutes:seconds.
- NODELIST指明了这个job在那个节点上运行，如果该job处于PD状态，则这里会指出该job处于PD状态的原因。主要的原因有：Resources，Priority

The squeue command has many options to easily let you view the information of interest to you in whatever format you prefer. See the man page for more information.

##### scontrol
我们可以用scontrol来获得更多的，关于节点，分区，job，job step(are sets of(possibly parallel) tasks within a job)和configuration的信息。也可以被管理员用来调整configuration。下面是几个scontrol的例子：

adev0: scontrol show partition
PartitionName=debug TotalNodes=5 TotalCPUs=40 RootOnly=NO
   Default=YES OverSubscribe=FORCE:4 PriorityTier=1 State=UP
   MaxTime=00:30:00 Hidden=NO
   MinNodes=1 MaxNodes=26 DisableRootJobs=NO AllowGroups=ALL
   Nodes=adev[1-5] NodeIndices=0-4

PartitionName=batch TotalNodes=10 TotalCPUs=80 RootOnly=NO
   Default=NO OverSubscribe=FORCE:4 PriorityTier=1 State=UP
   MaxTime=16:00:00 Hidden=NO
   MinNodes=1 MaxNodes=26 DisableRootJobs=NO AllowGroups=ALL
   Nodes=adev[6-15] NodeIndices=5-14


adev0: scontrol show node adev1
NodeName=adev1 State=DOWN* CPUs=8 AllocCPUs=0
   RealMemory=4000 TmpDisk=0
   Sockets=2 Cores=4 Threads=1 Weight=1 Features=intel
   Reason=Not responding [slurm@06/02-14:01:24]

65648     batch  math  phil PD  0:00     6 (Resources)
adev0: scontrol show job
JobId=65672 UserId=phil(5136) GroupId=phil(5136)
   Name=math
   Priority=4294901603 Partition=batch BatchFlag=1
   AllocNode:Sid=adev0:16726 TimeLimit=00:10:00 ExitCode=0:0
   StartTime=06/02-15:27:11 EndTime=06/02-15:37:11
   JobState=PENDING NodeList=(null) NodeListIndices=
   NumCPUs=24 ReqNodes=1 ReqS:C:T=1-65535:1-65535:1-65535
   OverSubscribe=1 Contiguous=0 CPUs/task=0 Licenses=(null)
   MinCPUs=1 MinSockets=1 MinCores=1 MinThreads=1
   MinMemory=0 MinTmpDisk=0 Features=(null)
   Dependency=(null) Account=(null) Requeue=1
   Reason=None Network=(null)
   ReqNodeList=(null) ReqNodeListIndices=
   ExcNodeList=(null) ExcNodeListIndices=
   SubmitTime=06/02-15:27:11 SuspendTime=None PreSusTime=0
   Command=/home/phil/math
   WorkDir=/home/phil

##### srun
通过srun，我们通过一句指令登记job step中的多个任务(task)，并为他们分配资源。(It is possible to create a resource allocation and launch the tasks for a job step in a single command line using the srun command.)<br>
这里我们再三个节点上(-N3)执行'/bin/hostname'命令，并把task号码加到对应的输出上一起输出(-l)。<br>
默认情况下，我们会使用默认的分区，并且每个节点上运行一个task。<br>
Note that the srun command has many options available to control what resource are allocated and how tasks are distributed across those resources.

adev0: srun -N3 -l /bin/hostname
0: adev3
1: adev4
2: adev5

##### other
一种常用的操作是提交一个script，然后以后再进行运行。这里我们要提交的脚本叫my.script。同时我们通过-w指定要用哪个节点进行运行(-w "adev[9-10])、通过-4指明我们的job step会长生4个tasks，从而确保该任务至少能够分配到4个处理器、通过-o我们指明了输出文件的位置(-o my.stdout)。<br>
值的注意的是，在脚本中，我们还为这个job设置了时间限制(timelimit具体不知道是干什么用的)。正如这里所展示的，通过前缀'#SBATCH'，我们可以在脚本中为我们的命令添加选项。值得提一下的是，
1. 应该在执行其他正式命令语句之前添加选项的行为，否则无效。
2. 如果这里添加的选项和命令行中的选项重合了，那么这里的选项将会被覆盖掉。

adev0: cat my.script
#!/bin/sh
#SBATCH --time=1
/bin/hostname
srun -l /bin/hostname
srun -l /bin/pwd

adev0: sbatch -n4 -w "adev[9-10]" -o my.stdout my.script
sbatch: Submitted batch job 469

adev0: cat my.stdout
adev9
0: adev9
1: adev9
2: adev10
3: adev10
0: /home/jette
1: /home/jette
2: /home/jette
3: /home/jette

# 剩下一点没有看懂，就跳了，是不是太任性了

## 定制slurm
SLURM 并不是一个静态的资源管理器，而是一个可以结合新行为的高度动态的资源管理器。SLURM 实现了一个插件应用程序编程接口 (API)，允许运行时库在运行时动态加载。这个 API 已经用于开发各种新行为，包括互连结构、身份验证和调度。插件接口支持各种其他功能，比如作业统计、加密功能、消息传递接口 (MPI)、过程跟踪以及资源选择。所有这些都允许 SLURM 可以轻松支持不同的集群架构和实现。请查看 参考资料 部分的 SLURM 程序员指南，以了解详细信息。